In [1]:
import numpy as np
from scipy.stats import norm

def BS_call(S, K, r, T, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def BS_vega(S, K, r, T, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    return S*np.sqrt(T)*norm.pdf(d1)

def BS_call_iv(S, K, r, T, v_guess, c_actual):
    max_iter = 100
    precision = 1.0e-10
    
    for i in range(0, max_iter):
        c = BS_call(S, K, r, T, v_guess)
        vega = BS_vega(S, K, r, T, v_guess)
        diff = c - c_actual
        
        print('*** This is loop #'+str(i)+' ***')
        print('The guess for volatility is ' + str(round(v_guess,3)) + 
              ' and the call price is ' + str(round(c,3)))
        
        if (abs(diff) < precision):
            return v_guess
        v_guess = v_guess - diff/vega

    return v_guess  # if max iterations reached

# Given parameters from your problem
S = 150
K = 145
r = 0.01
T = 2
c_actual = 20.658

# Initial volatility guess
v_guess = 0.5

print('The market price is ' + str(round(c_actual,3)))
implied_vol = BS_call_iv(S, K, r, T, v_guess, c_actual)
print('Final implied volatility value: ' + str(round(implied_vol,4)))

# Verify the result
final_price = BS_call(S, K, r, T, implied_vol)
print('\nVerification:')
print('Market price:', round(c_actual,6))
print('Price using calculated implied volatility:', round(final_price,6))
print('Difference:', abs(c_actual - final_price))

The market price is 20.658
*** This is loop #0 ***
The guess for volatility is 0.5 and the call price is 44.409
*** This is loop #1 ***
The guess for volatility is 0.192 and the call price is 20.032
*** This is loop #2 ***
The guess for volatility is 0.2 and the call price is 20.658
*** This is loop #3 ***
The guess for volatility is 0.2 and the call price is 20.658
Final implied volatility value: 0.2

Verification:
Market price: 20.658
Price using calculated implied volatility: 20.658
Difference: 2.9970692594361026e-11


In [2]:
import numpy as np

# Set parameters
S1_0 = 50  # Initial price of stock 1
S2_0 = 40  # Initial price of stock 2
sigma1 = 0.3  # Volatility of stock 1
sigma2 = 0.25  # Volatility of stock 2
rho = -0.5  # Correlation between stocks
K = 45  # Strike price
r = 0.05  # Risk-free rate
T = 1  # Time to maturity
nstep = 1  # Number of time steps
npaths = 10000  # Number of simulation paths

# Set random seed
np.random.seed(5)

# Generate correlated normal random variables
z1 = np.random.standard_normal(npaths)
z2 = rho * z1 + np.sqrt(1 - rho**2) * np.random.standard_normal(npaths)

# Calculate stock prices at maturity using geometric Brownian motion
dt = T/nstep
S1_T = S1_0 * np.exp((r - 0.5 * sigma1**2) * T + sigma1 * np.sqrt(T) * z1)
S2_T = S2_0 * np.exp((r - 0.5 * sigma2**2) * T + sigma2 * np.sqrt(T) * z2)

# Calculate payoffs
payoffs = np.maximum(np.maximum(S1_T, S2_T) - K, 0)

# Calculate option price
option_price = np.exp(-r * T) * np.mean(payoffs)

print(f"Maximum of Maximum Exchange Option Price: {option_price:.4f}")

# Additional statistics for verification
print("\nVerification Statistics:")
print(f"Mean of S1_T: {np.mean(S1_T):.4f}")
print(f"Mean of S2_T: {np.mean(S2_T):.4f}")
print(f"Std of S1_T: {np.std(S1_T):.4f}")
print(f"Std of S2_T: {np.std(S2_T):.4f}")
print(f"Correlation between S1_T and S2_T: {np.corrcoef(S1_T, S2_T)[0,1]:.4f}")

Maximum of Maximum Exchange Option Price: 12.1592

Verification Statistics:
Mean of S1_T: 52.5536
Mean of S2_T: 42.0888
Std of S1_T: 16.0247
Std of S2_T: 10.7050
Correlation between S1_T and S2_T: -0.4683


In [6]:
import numpy as np
from scipy.stats import norm

# Black-Scholes formula for call and put options
def black_scholes_call(S, K, r, T, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    call = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    return call

def black_scholes_put(S, K, r, T, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    put = K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
    return put

# Function to calculate chooser option price
def chooser_option_price(S, K, r, t, T, sigma):
    """
    Price a chooser option using Black-Scholes formula
    
    Parameters:
    S: Current stock price
    K: Strike price
    r: Risk-free rate
    t: Time until choice must be made
    T: Time until option expiration
    sigma: Volatility
    
    Returns:
    float: Price of the chooser option
    """
    # Call price with maturity T
    call_price = black_scholes_call(S, K, r, T, sigma)
    # Put price with maturity T - t
    put_price = black_scholes_put(S, K, r, T - t, sigma)
    # Chooser option price
    chooser_price = call_price + put_price
    return chooser_price

# Given parameters
S = 40      # Current stock price
K = 50      # Strike price
r = 0.02    # Risk-free rate
t = 1       # Time until choice
T = 1.5     # Time until expiration
sigma = 0.2 # Volatility

# Calculate chooser option price
chooser_price = chooser_option_price(S, K, r, t, T, sigma)
call_price = black_scholes_call(S, K, r, T, sigma)
put_price = black_scholes_put(S, K, r, T - t, sigma)

# Print results
print(f"Chooser Option Price: {chooser_price:.4f}")
print("\nFor comparison:")
print(f"European Call Price (T = {T}): {call_price:.4f}")
print(f"European Put Price (T - t = {T - t}): {put_price:.4f}")
print(f"Chooser Option = Call (T) + Put (T - t): {call_price + put_price:.4f}")


Chooser Option Price: 11.0015

For comparison:
European Call Price (T = 1.5): 1.3179
European Put Price (T - t = 0.5): 9.6836
Chooser Option = Call (T) + Put (T - t): 11.0015


In [5]:
import numpy as np
from scipy.stats import norm

# Black-Scholes formula for call and put options
def bs_call(S, K, r, T, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def bs_put(S, K, r, T, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

# Given parameters
S = 40       # Current stock price
K = 50       # Strike price
r = 0.02     # Risk-free rate
t = 1        # Choice time (in years)
T = 1.5      # Time to maturity (in years)
sigma = 0.2  # Volatility

# Price of the chooser option
chooser_option_price = bs_call(S, K, r, T, sigma) + bs_put(S, K, r, T - t, sigma)

# Print the result
print(f"The price of the chooser option is: {chooser_option_price:.4f}")


The price of the chooser option is: 11.0015


In [7]:
import numpy as np

# Parameters
R = 1.025  # Risk-free rate

# Stock prices at t=1
S_up_1 = 12.5
S_down_1 = 8.0

# Stock prices at t=2
S_up_2 = 15.625
S_middle_2 = 10.0
S_down_2 = 6.4

# Stock prices at t=3
S_up_3 = 19.53125
S_up_middle_3 = 12.5
S_down_middle_3 = 8.0
S_down_3 = 5.12

def calculate_probability(up_price, down_price, current_price, R):
    """Calculate risk-neutral probability"""
    return (R * current_price - down_price) / (up_price - down_price)

# Calculate probabilities at t=1 for the middle path
p_middle_1 = calculate_probability(S_up_2, S_down_2, S_middle_2, R)

print("Step by step calculation:")
print(f"For middle path at t=1:")
print(f"p = ({R} * {S_middle_2} - {S_down_2}) / ({S_up_2} - {S_down_2})")
print(f"p = ({R*S_middle_2} - {S_down_2}) / {S_up_2 - S_down_2}")
print(f"p = {(R*S_middle_2 - S_down_2):.4f} / {(S_up_2 - S_down_2):.4f}")
print(f"p = {p_middle_1:.4f}")

# Calculate total probability for m=1 path
# This is the probability of going down first (1-p) then up (p)
p_up_0 = calculate_probability(S_up_1, S_down_1, 10, R)
p_adjusted = (1 - p_up_0) * p_middle_1

print(f"\nInitial down probability (1-p) = {1-p_up_0:.4f}")
print(f"Second period up probability (p) = {p_middle_1:.4f}")
print(f"\nFinal adjusted risk-neutral probability Q~(m=1) = {p_adjusted:.4f}")

Step by step calculation:
For middle path at t=1:
p = (1.025 * 10.0 - 6.4) / (15.625 - 6.4)
p = (10.25 - 6.4) / 9.225
p = 3.8500 / 9.2250
p = 0.4173

Initial down probability (1-p) = 0.5000
Second period up probability (p) = 0.4173

Final adjusted risk-neutral probability Q~(m=1) = 0.2087


In [10]:
# Parameters
S0 = 10
R = 1.025
u = 12.5/10
d = 8/10

# First step: probability of going down (1-p)
p_first = (R - d)/(u - d)
prob_down = 1 - p_first

# Second step: probability of going up from down position
u2 = 10/8    # up factor from 8 to 10
d2 = 6.4/8   # down factor from 8 to 6.4
p_second = (R - d2)/(u2 - d2)

# Final probability is product of down then up
Q_tilde_m1 = prob_down * p_second

print(f"Adjusted risk-neutral probability Q~(m=1): {Q_tilde_m1:.4f}")

Adjusted risk-neutral probability Q~(m=1): 0.2500


In [11]:
import numpy as np

# Parameters
R = 1.025  # Risk-free rate

# Stock prices at t=1
S_up_1 = 12.5
S_down_1 = 8.0

# Stock prices at t=2
S_up_2 = 15.625
S_middle_2 = 10.0
S_down_2 = 6.4

def calculate_probability(up_price, down_price, current_price, R):
    """Calculate risk-neutral probability"""
    return (R * current_price - down_price) / (up_price - down_price)

# Calculate risk-neutral probability at t=0
p_up_0 = calculate_probability(S_up_1, S_down_1, 10, R)

# At t=1, the middle node's probability is still p_up_0 due to the symmetric tree
p_middle_1 = p_up_0

# Calculate total probability for m=1 path
# This is the probability of going down first (1-p) then up (p)
p_adjusted = (1 - p_up_0) * p_middle_1

print("Step-by-step calculation:")
print(f"Risk-neutral probability at t=0 (up move): p = {p_up_0:.4f}")
print(f"Risk-neutral probability for middle path at t=1: p = {p_middle_1:.4f}")
print(f"Probability of going down first (1-p): {1-p_up_0:.4f}")
print(f"Probability of going up second (p): {p_middle_1:.4f}")
print(f"\nFinal adjusted risk-neutral probability Q~(m=1) = {p_adjusted:.4f}")


Step-by-step calculation:
Risk-neutral probability at t=0 (up move): p = 0.5000
Risk-neutral probability for middle path at t=1: p = 0.5000
Probability of going down first (1-p): 0.5000
Probability of going up second (p): 0.5000

Final adjusted risk-neutral probability Q~(m=1) = 0.2500


In [12]:
import numpy as np

# Set parameters
np.random.seed(1)
S0 = 100      # Initial stock price
K = 100       # Strike price
r = 0.05      # Risk-free rate
sigma = 0.2   # Volatility
T = 2         # Time to maturity
nsim = 10000  # Number of simulations
nstep = 1     # Number of time steps
h = 0.0001    # Bump size for vega calculation

def monte_carlo_call(S0, K, r, sigma, T, nsim, nstep):
    """
    Price a European call option using Monte Carlo simulation
    """
    dt = T/nstep
    # Generate random normal variables
    Z = np.random.standard_normal(nsim)
    
    # Calculate terminal stock prices
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    
    # Calculate call payoffs
    payoffs = np.maximum(ST - K, 0)
    
    # Discount payoffs
    price = np.exp(-r * T) * np.mean(payoffs)
    return price

# Calculate option prices with bumped volatilities
price_up = monte_carlo_call(S0, K, r, sigma + h, T, nsim, nstep)
price_down = monte_carlo_call(S0, K, r, sigma - h, T, nsim, nstep)

# Calculate vega using central difference
vega = (price_up - price_down) / (2 * h)

# Calculate base price for verification
base_price = monte_carlo_call(S0, K, r, sigma, T, nsim, nstep)

print(f"European Call Option Price: {base_price:.4f}")
print(f"Price with sigma + h: {price_up:.4f}")
print(f"Price with sigma - h: {price_down:.4f}")
print(f"Vega: {vega:.4f}")

# Additional verification statistics
print("\nVerification Statistics:")
print(f"Number of simulations: {nsim}")
print(f"Bump size (h): {h}")
print(f"Change in price for +h: {price_up - base_price:.6f}")
print(f"Change in price for -h: {base_price - price_down:.6f}")

European Call Option Price: 15.7845
Price with sigma + h: 16.2988
Price with sigma - h: 16.3456
Vega: -233.7098

Verification Statistics:
Number of simulations: 10000
Bump size (h): 0.0001
Change in price for +h: 0.514305
Change in price for -h: -0.561046


In [17]:
import numpy as np

# Parameters
np.random.seed(1)  # Seed for reproducibility
S_0 = 100          # Initial stock price
K = 100            # Strike price
r = 0.05           # Risk-free rate
sigma = 0.2        # Volatility
T = 2              # Time to maturity
nsim = 10000       # Number of Monte Carlo simulations
h = 0.0001         # Small change in volatility

# Monte Carlo function to compute option price
def monte_carlo_call_price(S_0, K, r, sigma, T, nsim):
    Z = np.random.normal(0, 1, nsim)  # Standard normal random variables
    S_T = S_0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)  # Terminal prices
    payoff = np.maximum(S_T - K, 0)  # Call option payoff
    price = np.exp(-r * T) * np.mean(payoff)  # Discounted expected payoff
    return price

# Compute option prices
price_at_sigma = monte_carlo_call_price(S_0, K, r, sigma, T, nsim)
price_at_sigma_plus_h = monte_carlo_call_price(S_0, K, r, sigma + h, T, nsim)

# Compute Vega
vega = (price_at_sigma_plus_h - price_at_sigma) / h

# Output results
print(f"Option price at sigma: {price_at_sigma:.6f}")
print(f"Option price at sigma + h: {price_at_sigma_plus_h:.6f}")
print(f"Vega of the European call option: {vega:.6f}")


Option price at sigma: 16.293790
Option price at sigma + h: 16.355756
Vega of the European call option: 619.662754


In [18]:
import numpy as np
import math as m

def BinoAmerCall(S, K, v, r, n, T, div):
    # Time step size
    dt = T / n

    # Up and down factors
    u = m.exp(v * m.sqrt(dt))
    d = 1 / u
    q = (m.exp((r - div) * dt) - d) / (u - d)

    # Initialize stock price tree
    Tree = np.zeros((n + 1, n + 1))
    for j in range(n + 1):
        for i in range(j + 1):
            Tree[i, j] = S * m.pow(d, i) * m.pow(u, j - i)

    # Call Tree 
    Call = np.zeros((n + 1, n + 1))
    Exer = np.zeros((n + 1, n + 1))
    
    for j in range(n + 1, 0, -1):
        for i in range(j):
            
            if j == n + 1:
                Call[i, j - 1] = max(Tree[i, j - 1] - K, 0)  # Call payoff: St - K 
            
            else:
                Call[i, j - 1] = max(Tree[i, j - 1] - K,  # early exercise value
                                   m.exp(-r * dt) * (q * Call[i, j] + (1 - q) * Call[i + 1, j]))  # continuation value
            
                if Tree[i, j - 1] - K > m.exp(-r * dt) * (q * Call[i, j] + (1 - q) * Call[i + 1, j]): 
                    Exer[i, j - 1] = 1 

    # Print trees
    print('Stock Price Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Tree, 2)))
    print('-------------------------------------------')
    print('Option Price Tree (n = ' + str(n) + '):\n', np.matrix(np.round(Call, 2)))
    print('-------------------------------------------')
    
    return Call[0, 0]

# Parameters
S0 = 100      # Initial stock price
K = 105       # Strike price
sigma = 0.4   # Volatility
r = 0.05      # Risk-free rate
T = 1         # Time to maturity
n = 5         # Number of steps
div = 0.03    # Dividend yield

# Calculate option price
price = BinoAmerCall(S0, K, sigma, r, n, T, div)

print(f"\nParameters:")
print(f"S0 = ${S0}")
print(f"K = ${K}")
print(f"σ = {sigma}")
print(f"r = {r}")
print(f"T = {T}")
print(f"n = {n}")
print(f"δ = {div}")

print(f"\nAmerican Call Option Price: ${price:.4f}")

# Calculate model parameters for verification
dt = T/n
u = m.exp(sigma * m.sqrt(dt))
d = 1/u
q = (m.exp((r - div) * dt) - d)/(u - d)

print("\nModel Parameters:")
print(f"dt = {dt:.4f}")
print(f"u = {u:.4f}")
print(f"d = {d:.4f}")
print(f"q = {q:.4f}")

Stock Price Tree (n = 5):
 [[100.   119.59 143.01 171.03 204.53 244.59]
 [  0.    83.62 100.   119.59 143.01 171.03]
 [  0.     0.    69.92  83.62 100.   119.59]
 [  0.     0.     0.    58.47  69.92  83.62]
 [  0.     0.     0.     0.    48.89  58.47]
 [  0.     0.     0.     0.     0.    40.88]]
-------------------------------------------
Option Price Tree (n = 5):
 [[ 14.89  25.33  41.75  66.15  99.53 139.59]
 [  0.     6.04  11.44  21.2   38.2   66.03]
 [  0.     0.     1.44   3.11   6.74  14.59]
 [  0.     0.     0.     0.     0.     0.  ]
 [  0.     0.     0.     0.     0.     0.  ]
 [  0.     0.     0.     0.     0.     0.  ]]
-------------------------------------------

Parameters:
S0 = $100
K = $105
σ = 0.4
r = 0.05
T = 1
n = 5
δ = 0.03

American Call Option Price: $14.8900

Model Parameters:
dt = 0.2000
u = 1.1959
d = 0.8362
q = 0.4665


In [21]:
import numpy as np
from scipy.stats import norm

# Black-Scholes-Merton formula for European put option
def black_scholes_put(S0, K, r, T, sigma):
    """
    Computes the price of a European put option using the Black-Scholes-Merton formula.
    
    Parameters:
        S0 (float): Current stock price
        K (float): Strike price
        r (float): Risk-free interest rate
        T (float): Time to maturity (in years)
        sigma (float): Volatility of the stock (standard deviation of returns)
        
    Returns:
        float: Price of the European put option
    """
    # Calculate d1 and d2
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Compute the put price using Black-Scholes-Merton formula
    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    
    return put_price

# Parameters
S0 = 80        # Current stock price
K = 65         # Strike price
r = 0.05       # Risk-free interest rate
T = 0.75       # Time to maturity (in years)
sigma = 0.5    # Volatility

# Compute the price of the European put option
put_price = black_scholes_put(S0, K, r, T, sigma)

# Output the result
print(f"The price of the European put option is: {put_price:.4f}")


The price of the European put option is: 5.3924


In [23]:
import numpy as np
from scipy.stats import norm

def price_forward_start_call(S0, alpha, r, t, T, sigma):
    """
    Calculate the price of a forward start call option using Black-Scholes-Merton formula
    
    Parameters:
    S0 (float): Initial stock price
    alpha (float): Strike price proportion
    r (float): Risk-free interest rate
    t (float): Forward start time
    T (float): Option maturity time
    sigma (float): Volatility
    
    Returns:
    float: Forward start call option price
    """
    # Calculate forward price at time t
    St = S0 * np.exp(r * t)  # Forward price at time t
    K = alpha * St  # Strike price will be alpha * St
    
    # Calculate d1 and d2 using the formula from the image
    tau = T - t  # Time to maturity from forward start date
    
    d1 = (1 / (sigma * np.sqrt(tau))) * (np.log(St/K) + (r + sigma**2/2) * tau)
    d2 = d1 - sigma * np.sqrt(tau)
    
    # Calculate option price using BSM formula
    call_price = St * norm.cdf(d1) - K * np.exp(-r * tau) * norm.cdf(d2)
    
    # Discount back to time 0
    call_price_t0 = call_price * np.exp(-r * t)
    
    return call_price_t0

# Example usage with given parameters
S0 = 5  # Initial stock price
alpha = 0.6  # Strike price proportion
r = 0.03  # Risk-free rate
t = 0.5  # Forward start time
T = 1.5  # Option maturity
sigma = 0.25  # Volatility

option_price = price_forward_start_call(S0, alpha, r, t, T, sigma)
print(f"Forward start call option price at t=0: {option_price:.4f}")

# Print intermediate calculations for verification
St = S0 * np.exp(r * t)
K = alpha * St
tau = T - t
d1 = (1 / (sigma * np.sqrt(tau))) * (np.log(St/K) + (r + sigma**2/2) * tau)
d2 = d1 - sigma * np.sqrt(tau)

print("\nIntermediate calculations:")
print(f"St (Forward price at t): {St:.4f}")
print(f"K (Strike price): {K:.4f}")
print(f"d1: {d1:.4f}")
print(f"d2: {d2:.4f}")

Forward start call option price at t=0: 2.0938

Intermediate calculations:
St (Forward price at t): 5.0756
K (Strike price): 3.0453
d1: 2.2883
d2: 2.0383


In [24]:
import numpy as np
from scipy.stats import norm

def forward_start_call(S0, alpha, r, T, t, sigma):
    """
    Compute the price of a forward start call option using the Black-Scholes-Merton method.
    
    Parameters:
        S0 (float): Current stock price at t=0
        alpha (float): Strike price fraction (K = alpha * S_t)
        r (float): Risk-free rate (continuously compounded)
        T (float): Maturity time of the option
        t (float): Time when the strike price is set
        sigma (float): Volatility of the stock
        
    Returns:
        float: Price of the forward start call option
    """
    # Effective maturity
    T_effective = T - t

    # Calculate d1 and d2
    d1 = (np.log(1 / alpha) + (r + 0.5 * sigma**2) * T_effective) / (sigma * np.sqrt(T_effective))
    d2 = d1 - sigma * np.sqrt(T_effective)

    # Compute option price
    term1 = S0 * norm.cdf(d1)
    term2 = S0 * np.exp(-r * T) * alpha * norm.cdf(d2)
    price = term1 - term2

    return price

# Parameters
S0 = 5         # Current stock price
alpha = 0.6    # Strike price fraction
r = 0.03       # Risk-free rate
t = 0.5        # Time when the strike is set
T = 1.5        # Option maturity
sigma = 0.25   # Volatility

# Compute the forward start call option price
price = forward_start_call(S0, alpha, r, T, t, sigma)

# Output the result
print(f"The price of the forward start call option is: {price:.4f}")


The price of the forward start call option is: 2.1362


In [25]:
import numpy as np

# Parameters
np.random.seed(5)
nsim = 50000  # Number of independent paths
nstep = 250   # Number of time steps
T = 1         # Time to maturity
dt = T / nstep  # Time step size
r = 0.04      # Risk-free rate
sigma = 0.35  # Volatility
S_0 = 110     # Initial stock price
H = 160       # Barrier level
K = 120       # Strike price

# Monte Carlo simulation
def price_knock_out_call(S_0, H, K, r, sigma, T, dt, nstep, nsim):
    """
    Price a knock-out call option using Monte Carlo with antithetic variates.
    
    Parameters:
        S_0 (float): Initial stock price
        H (float): Barrier level
        K (float): Strike price
        r (float): Risk-free rate
        sigma (float): Volatility
        T (float): Time to maturity
        dt (float): Time step size
        nstep (int): Number of time steps
        nsim (int): Number of independent simulations
    
    Returns:
        float: Price of the knock-out call option
    """
    # Simulate Brownian increments
    Z = np.random.normal(0, 1, size=(nsim, nstep))
    Z_antithetic = -Z  # Antithetic variates

    # Combine paths for standard and antithetic variates
    all_Z = np.vstack((Z, Z_antithetic))
    total_paths = all_Z.shape[0]

    # Simulate paths
    S_paths = np.zeros((total_paths, nstep + 1))
    S_paths[:, 0] = S_0
    for t in range(1, nstep + 1):
        S_paths[:, t] = S_paths[:, t - 1] * np.exp(
            (r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * all_Z[:, t - 1]
        )

    # Check for barrier breach
    barrier_breached = np.any(S_paths > H, axis=1)

    # Payoff calculation for paths that do not breach the barrier
    S_T = S_paths[:, -1]
    payoff = np.where(barrier_breached, 0, np.maximum(S_T - K, 0))

    # Discounted average payoff
    discounted_payoff = np.exp(-r * T) * payoff
    price = np.mean(discounted_payoff)

    return price

# Price the knock-out call option
knock_out_call_price = price_knock_out_call(S_0, H, K, r, sigma, T, dt, nstep, nsim)

# Output the result
print(f"The price of the knock-out call option is: {knock_out_call_price:.4f}")


The price of the knock-out call option is: 1.9725


In [26]:
import numpy as np
from scipy.stats import norm

# Set parameters
np.random.seed(5)
nsim = 50000  # Number of simulations (will be doubled due to antithetic variates)
nstep = 250   # Number of time steps
T = 1         # Time to maturity
dt = T/nstep  # Time step size
r = 0.04      # Risk-free rate
sigma = 0.35  # Volatility
S_0 = 110     # Initial stock price
K = 120       # Strike price
H = 160       # Barrier level

# Generate random normal variables for paths
Z = np.random.normal(0, 1, size=(nsim, nstep))

# Create antithetic variates
Z_antithetic = -Z

# Combine original and antithetic variates
Z_combined = np.vstack([Z, Z_antithetic])

# Initialize arrays for stock prices
S = np.zeros((2*nsim, nstep + 1))
S[:, 0] = S_0

# Simulate stock price paths
for t in range(nstep):
    S[:, t+1] = S[:, t] * np.exp((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z_combined[:, t])

# Check for barrier hits and calculate payoffs
barrier_hit = np.any(S >= H, axis=1)
final_prices = S[:, -1]
payoffs = np.maximum(final_prices - K, 0)
payoffs[barrier_hit] = 0  # Set payoff to zero if barrier was hit

# Calculate option price
option_price = np.mean(payoffs) * np.exp(-r*T)
std_error = np.std(payoffs) * np.exp(-r*T) / np.sqrt(2*nsim)

# Print results
print(f"Knock-out Call Option Price: {option_price:.4f}")
print(f"Standard Error: {std_error:.4f}")

Knock-out Call Option Price: 1.9725
Standard Error: 0.0186


In [27]:
import numpy as np
from scipy.stats import norm

def black_scholes_call(S, K, r, T, sigma):
    """
    Compute the price of a European call option using the Black-Scholes formula.
    
    Parameters:
        S (float): Current stock price
        K (float): Strike price
        r (float): Risk-free rate
        T (float): Time to maturity
        sigma (float): Volatility of the stock
    
    Returns:
        float: Price of the European call option
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

# Given parameters
S0 = 5         # Current stock price
r = 0.02       # Risk-free rate
T = 1          # Time to maturity
sigma = 0.35   # Volatility
K1 = 2         # Strike price of long call
K2 = 8         # Strike price of short call

# Price the two options using Black-Scholes
call_price_long = black_scholes_call(S0, K1, r, T, sigma)  # Long call
call_price_short = black_scholes_call(S0, K2, r, T, sigma) # Short call

# Net price of the bull call spread
bull_call_spread_price = call_price_long - call_price_short

# Output the result
print(f"Price of the long call (K1 = {K1}): {call_price_long:.4f}")
print(f"Price of the short call (K2 = {K2}): {call_price_short:.4f}")
print(f"Price of the product (Bull Call Spread): {bull_call_spread_price:.4f}")


Price of the long call (K1 = 2): 3.0408
Price of the short call (K2 = 8): 0.1018
Price of the product (Bull Call Spread): 2.9391


In [31]:
import numpy as np
from scipy.stats import norm

def bsm_call_price(S, K, T, r, sigma):
    """Calculate BSM European call option price"""
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def bsm_digital_call_price(S, K, T, r, sigma, payout):
    """Calculate BSM Digital call option price"""
    d2 = (np.log(S/K) + (r - sigma**2/2)*T) / (sigma*np.sqrt(T))
    return payout * np.exp(-r*T) * norm.cdf(d2)

# Given parameters
S0 = 5      # Current stock price
r = 0.02    # Risk-free rate
T = 1       # Time to maturity
sigma = 0.35  # Volatility

# Components of the payoff
# 1. Digital Call at K=2 with payout=2
digital_call_2 = bsm_digital_call_price(S0, 2, T, r, sigma, 2)

# 2. European Call at K=2
euro_call_2 = bsm_call_price(S0, 2, T, r, sigma)

# 3. Negative Digital Call at K=8 with payout=2
digital_call_8 = bsm_digital_call_price(S0, 8, T, r, sigma, 2)

# Total price
total_price = digital_call_2 + euro_call_2 - digital_call_8

print(f"""BSM Parameters:
S0 = {S0}
r = {r}
T = {T}
sigma = {sigma}

Price Components:
1. Digital Call (K=2, payout=2): {digital_call_2:.4f}
2. European Call (K=2): {euro_call_2:.4f}
3. Digital Call (K=8, payout=2): {digital_call_8:.4f}

Total Price: {total_price:.4f}""")


BSM Parameters:
S0 = 5
r = 0.02
T = 1
sigma = 0.35

Price Components:
1. Digital Call (K=2, payout=2): 1.9482
2. European Call (K=2): 3.0408
3. Digital Call (K=8, payout=2): 0.1412

Total Price: 4.8478


In [33]:
import numpy as np
from scipy.stats import norm

def asset_or_nothing_call(S, K, r, T, sigma):
    """
    Price an asset-or-nothing call option using the Black-Scholes formula.
    Parameters:
        S (float): Current stock price
        K (float): Strike price
        r (float): Risk-free rate
        T (float): Time to maturity
        sigma (float): Volatility
    Returns:
        float: Price of the asset-or-nothing call option
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S * norm.cdf(d1)

# Given parameters
S0 = 5         # Current stock price
r = 0.02       # Risk-free rate
T = 1          # Time to maturity
sigma = 0.35   # Volatility
K1 = 2         # Strike price of the first asset-or-nothing call
K2 = 8         # Strike price of the second asset-or-nothing call

# Price each component
price_asset_or_nothing_call_2 = asset_or_nothing_call(S0, K1, r, T, sigma)
price_asset_or_nothing_call_8 = asset_or_nothing_call(S0, K2, r, T, sigma)

# Combine the components
product_price = price_asset_or_nothing_call_2 - price_asset_or_nothing_call_8

# Output the results
print(f"Price of Asset-or-Nothing Call at K1 = {K1}: {price_asset_or_nothing_call_2:.4f}")
print(f"Price of Asset-or-Nothing Call at K2 = {K2}: {price_asset_or_nothing_call_8:.4f}")
print(f"Price of the Product: {product_price:.4f}")


Price of Asset-or-Nothing Call at K1 = 2: 4.9891
Price of Asset-or-Nothing Call at K2 = 8: 0.6667
Price of the Product: 4.3224


In [3]:
import numpy as np
from scipy.stats import norm

def bsm_call_price(S0, K, r, sigma, T):
    """
    Price a European call option under the Black-Scholes-Merton model.
    """
    if S0 == K:
        return S0 * np.exp(-r * T)
    elif S0 == 0 or K == 0:
        return 0
    else:
        d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Parameters
S0 = 5         # Current stock price
r = 0          # Risk-free rate
sigma = 0.35   # Volatility
T = 1          # Time to maturity (1 year)
call_price = 1 # One-year ATM European call option price

# Calculate the cost of constructing the payoff
cost = 0
for i in range(0, 21):
    if i < 10:
        cost += (50 - i * 5) * call_price
    else:
        cost += (-10 + (i - 10) * 1) * call_price

print(f"Cost of constructing the payoff: {cost:.4f}")

Cost of constructing the payoff: 220.0000
